In [1]:
from cntools.datasets import load_ct_order, prep_t2d, Dataset
from cntools.identification import CC, CFIDF, CNE, Spatial_LDA
from cntools.smoothing import NaiveSmooth, HMRF
from cntools.utils import cns_info
import os
import pandas as pd
import pickle

In [2]:
# Download T2D data
if not os.path.exists('data/T2D'):
    os.system('gdown --folder https://drive.google.com/drive/folders/1eC3ahSIzCZ5vq_bynSl3IX7V9w_bQgg8?usp=share_link')

In [3]:
# Load dataset
df = prep_t2d(pd.read_csv('data/T2D/Cell-ID_by-islet.csv'))
ct_order = load_ct_order('data/T2D/ct_order.json')
ds = Dataset(df, ct_order)

In [ ]:
# Save processed dataframe and dataset object
df.to_csv('data/T2D/T2D_df.csv', index=False)
pickle.dump(ds, open(f'data/T2D/T2D_ds.pkl', 'wb'))

In [ ]:
# identifier = CC(n_cns=6, m=5)
# identifier = CFIDF(n_cns=6, eps=57, r=1.0)
identifier = CNE(n_cns=6, perp=15, lam=0.25)
# identifier = Spatial_LDA(n_cns=6, eps=100, b=0.25, train_size_fraction=0.989)

cns = identifier.fit(ds)

In [ ]:
smoother = NaiveSmooth(ds=ds, n_cns=identifier.n_cns, feats=identifier.feats, s=3)
cns_smoothed_naive = smoother.fit(cns)

In [ ]:
smoother = HMRF(ds=ds, n_cns=identifier.n_cns, eps=45, beta=9)
cns_smoothed_hmrf = smoother.fit(cns)

In [7]:
out_dir = 'cn/T2D/CNE'
os.makedirs(out_dir, exist_ok=True)
pickle.dump(cns, open(f'{out_dir}/cns_n_cns=6_perp=15_lam=0.25.pkl', 'wb'))
pickle.dump(cns_smoothed_naive, open(f'{out_dir}/cns_n_cns=6_perp=15_lam=0.25_naive.pkl', 'wb'))
pickle.dump(cns_smoothed_hmrf, open(f'{out_dir}/cns_n_cns=6_perp=15_lam=0.25_hmrf.pkl', 'wb'))